<div class="alert alert-danger">

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Automatic filling of Hamish Hodder's Stock Analysis Spreadsheet </div>

Author     : Willy LAO

Last update: 30/07/2020

</div>

**Ressources:**

Google Spreadsheet API: https://www.youtube.com/watch?v=sAgWCbGMzTo

Selenium: 
- https://www.scrapingbee.com/blog/practical-xpath-for-web-scraping/
- https://1www.scrapingbee.com/blog/selenium-python/

Hamish Hodder's Analysis Spreadsheet: https://www.youtube.com/watch?v=P5sICGKnpwE&t=459s

Capital expenditure: https://www.koyfin.com/company/AAPL/capex

<div class="alert alert-warning">

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Before running this notebook </div>

- Download the last version of Hamish Hodder's Analysis spreadsheet (see link)
- Create a copy and rename it with the name of the stock to analyze (expl: 'Boeing')
- Enable the Google API and the Google Spreadsheet API (see link)
- Add Credentials and download the json file of the API
- Put it in the same folder as this notebook
- Update the information below

</div>

In [21]:
## TO UPDATE ####################################################

ticker_symbol  = 'AAPL'                                         # Symbol of the stock 
quick_fs_url   = "https://quickfs.net/company/AAPL"             # URL on QuickFS
spreadsheet_id = '1mpUfmnTC3KOCbxAnDCMCpNdqD0bWrTZSMzL0xcQfhvQ' # ID of the Google spreasheet (to find in its link)

##################################################################

<div class="alert alert-info">
    
# Install and import the useful libraries

In [22]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

import time
import openpyxl
import pickle
import os
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request

<div class="alert alert-info">
    
# Define useful functions

In [23]:
###############################################################################################
# Purpose: Update the variable parameters_scrape

# Inputs : parameters_scrape  : contains the value of the parameters to scrape on QuickFS
#          parameter_selenium : contains the value of the parameters retrieved from QuickFS 
#          parameter          : name of the scraped parameter
#          financial_statement: Financial statement where the parameter was scraped

# Outputs: None

# Version: Python 3
###############################################################################################

def fill_parameters_scrape(parameters_scrape, parameter_selenium, parameter, financial_statement):
    # Scrape 10 years: 2010 - 2019
    parameters_list = []
    
    if financial_statement == 'Balance sheet': # The data begins at 2009 for the Balance sheet
        indices = np.arange(1,11)
    else:                                      # The data begins at 2010 for the Income statement and the Cash flow statement
        indices = np.arange(0,10)
        
    for i in indices:
        parameters_list.append(parameter_selenium[i].text)
    parameters_scrape[parameter] = parameters_list

In [24]:
###############################################################################################
# Purpose: Connect to a Google API

# Inputs : client_secret_file : json file to download from the Google API (see the tutorial)
#          api_name           : API used 
#          api_version        : Version of the API
#          scopes             : read, write... (see all the possibilities on Google API)

# Outputs: service: variable enabling to connect to the Google API

# Version: Python 3
###############################################################################################
 
def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)
 
    cred = None
 
    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)
 
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)
 
    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()
 
        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)
 
    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

In [25]:
###############################################################################################
# Purpose: Write in a Google Spreadsheet using the Google Spreadsheets API

# Inputs : service          : variable enabling to connect to the Google API
#          spreadsheet_id   : ID of the Google spreasheet (to find in its link)
#          values           : values to write in the spreadsheet - 2D array ([['blablabla']])
#          cell_range_insert: cell of reference from which we begin to write
#          worksheet_name   : name of the worksheet where the values will be written

# Outputs: None

# Version: Python 3
###############################################################################################

def write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name):
    value_range_body = {
    'majorDimension': 'ROWS',
    'values': values
    }

    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range=worksheet_name + cell_range_insert,
        body=value_range_body
    ).execute()

In [26]:
###########################################################################################################
# Purpose: Read a Google Spreadsheet using the Google Spreadsheets API

# Inputs : service        : variable enabling to connect to the Google API
#          spreadsheet_id : ID of the Google spreasheet (to find in its link)
#          cell_range_read: cell of reference from which we begin to read. Contains the worksheet_name too

# Outputs: None

# Version: Python 3
###########################################################################################################

def read_google_sheet(service, spreadsheet_id, cell_range_read):
    
    response = service.spreadsheets().values().get(
        spreadsheetId = spreadsheet_id,
        majorDimension = 'ROWS',
        range = cell_range_read
    ).execute()
    
    return(response)

<div class="alert alert-info">
    
# Scrape QuickFS

<div class="alert alert-danger">
    
<b>If the error `list out of range` appears, it means that the webpage did not have the time to load.

Please run the cell again.</b>

In [27]:
#### Options to configure the Chrome window opened by Selenium

DRIVER_PATH = '/home/willy/Willy taff/Investing/chromedriver' # Path of chromedriver (download it first)

# Options of the Chrome window that will open
options = Options()
options.headless = True                         # If true: a window is not open but runs in the background
options.add_argument("--window-size=1920,1200")

# Open the url with Chrome
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get(quick_fs_url)

# Wait a bit for the page to load
time.sleep(5)

In [28]:
#### Signing in is required in order to have access to all the data

# Click on the Login button
driver.find_element_by_xpath('//a[@routerlink="/login" and @id="header-content-loginBtn"]').click()

# Input the email
email_input = driver.find_element_by_xpath("//input[@type='email']")
email_input.send_keys('immortal.bourse@gmail.com')

# Input the password
password_input = driver.find_element_by_xpath("//input[@type='password']")
password_input.send_keys('Leobeedo101259')

# Click on the Sign in button
driver.find_element_by_xpath('//button[@type="submit" and @id="submitLoginFormBtn"]').click()

# Wait a bit for the page to load
time.sleep(5)

# Connect to the link again
driver.get(quick_fs_url)

In [29]:
#### Scrape the Income statement on QuickFS

# Dictionnary containing the values of the parameters to scrape
parameters_scrape = {}

# Click on the scrolling list
driver.find_element_by_xpath('//div[@class="dropdownLabel"]').click()

# Click on Income statement and wait
driver.find_element_by_xpath('//a[@id="is"]').click()
time.sleep(7)

# Scrape the Income statement 
revenue_selenium = driver.find_elements_by_xpath('//td[contains(text(), "Revenue")]/following-sibling::td')
eps_selenium = driver.find_elements_by_xpath('//td[contains(text(), "EPS (Diluted)")]/following-sibling::td')

# Update parameters_scrape
fill_parameters_scrape(parameters_scrape, revenue_selenium, 'Revenue', 'Income statement')
fill_parameters_scrape(parameters_scrape, eps_selenium, 'EPS (Diluted)', 'Income statement')

In [30]:
#### Scrape the Balance sheet on QuickFS

# Click on the scrolling list
driver.find_element_by_xpath('//div[@class="dropdownLabel"]').click()

# Click on the Balance sheet and wait
driver.find_element_by_xpath('//a[@id="bs"]').click()
time.sleep(7)

# Scrape the Balance sheet
total_current_assets_selenium      = driver.find_elements_by_xpath('//td[contains(text(), "Total Current Assets")]/following-sibling::td')
total_current_liabilities_selenium = driver.find_elements_by_xpath('//td[contains(text(), "Total Current Liabilities")]/following-sibling::td')
total_liabilities_selenium         = driver.find_elements_by_xpath('//td[contains(text(), "Total Liabilities")]/following-sibling::td')
shareholders_equity_selenium       = driver.find_elements_by_xpath('//td[contains(text(), "Shareholders\' Equity")]/following-sibling::td')
long_term_debt_selenium            = driver.find_elements_by_xpath('//td[contains(text(), "Long-Term Debt")]/following-sibling::td')

# Update parameters_scrape
fill_parameters_scrape(parameters_scrape, total_current_assets_selenium, 'Total Current Assets', 'Balance sheet')
fill_parameters_scrape(parameters_scrape, total_current_liabilities_selenium, 'Total Current Liabilities', 'Balance sheet')
fill_parameters_scrape(parameters_scrape, total_liabilities_selenium, 'Total Liabilities', 'Balance sheet')
fill_parameters_scrape(parameters_scrape, shareholders_equity_selenium, "Shareholders' Equity", 'Balance sheet')
fill_parameters_scrape(parameters_scrape, long_term_debt_selenium, "Long-Term Debt", 'Balance sheet')

In [31]:
#### Scrape the Cashflow statement on QuickFS

# Click on the scrolling list
driver.find_element_by_xpath('//div[@class="dropdownLabel"]').click()

# Click on the Cashflow statement and wait
driver.find_element_by_xpath('//a[@id="cf"]').click()
time.sleep(7)

# Scrape the Cashflow statement
cash_from_operations_selenium = driver.find_elements_by_xpath('//td[contains(text(), "Cash From Operations")]/following-sibling::td')

# Update parameters_scrape
fill_parameters_scrape(parameters_scrape, cash_from_operations_selenium, 'Cash From Operations', 'Cashflow statement')

# Quit the driver
driver.quit()

<div class="alert alert-info">
    
# Write in the Google Spreadsheet

<div class="alert alert-danger">
    
<b>Do not forget to enable Google API and Google Spreadsheet API.

You have to download the json file of the API and name it `client_secret.json` as well.</b>

In [32]:
#### Connect to the Google API and Google Spreadsheets API

CLIENT_SECRET_FILE = 'client_secret.json'
API_NAME = 'sheets'
API_VERSION = 'v4'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets'] # Read and write
 
# Create the service connecting to the Google API
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
 
# Connect to the Google Spreadsheets API
mySpreadsheets = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()

client_secret.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
sheets service created successfully


In [33]:
#### Write in the sheet DATA INPUT 
worksheet_name    = 'DATA INPUT!'   # Be careful of the "!" at the end of 'DATA INPUT!'

# Fill the cell of the Ticker symbol 'A2'
values = ([[ticker_symbol]])        # Be careful, needs to be a 2D array: ([['blablabla']])
cell_range_insert = 'A2'  
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name) 

# Fill the cells, columns 2 - 12
values = (
    (parameters_scrape['Revenue']),
    (parameters_scrape['EPS (Diluted)']),
    (parameters_scrape["Shareholders' Equity"]),
    (parameters_scrape["Cash From Operations"])
)
cell_range_insert = 'B5'
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name)
 
    
# Fill the cells (11,11), (12,11), (13,11)
values = (
    ([parameters_scrape['Total Current Assets'][-1]]),
    ([parameters_scrape['Total Current Liabilities'][-1]]),
    ([parameters_scrape['Total Liabilities'][-1]])
)
cell_range_insert = 'K11'
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name)

In [34]:
#### Write in the sheet ROIC
worksheet_name = 'ROIC!' # Be careful of the "!" at the end of 'ROIC!'

# Fill the cells, columns 2 - 12
values = (
    ([parameters_scrape['Long-Term Debt']]) # Be careful, needs to be a 2D array: ([['blablabla']])
)
cell_range_insert = 'B3'
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name)

In [35]:
#### Read the last Cash flow for owners and write it in the Discounted Cash Flow sheet
cell_range_read = 'DATA INPUT!K10'

response = read_google_sheet(service, spreadsheet_id, cell_range_read)
values = ([0, response['values'][0][0]],
          [0.02, response['values'][0][0]],
          [0.04, response['values'][0][0]],
          [0.06, response['values'][0][0]],
          [0.08, response['values'][0][0]],
         ) # Be careful, needs to be a 2D array: ([['blablabla']])

worksheet_name = 'Discounted Cash Flow!'
cell_range_insert = 'B2'
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name)

# Write 1% in the Risk-free rate
values = ([[0.01]])
cell_range_insert = 'B8'
write_google_sheet(service, spreadsheet_id, values, cell_range_insert, worksheet_name)

# OLD

In [36]:
# #### TO UPDATE

# stock = 'Airbus'

# ####

# file_name = stock + '.xlsx'
# workbook  = openpyxl.load_workbook("/home/willy/Willy taff/Investing/" + file_name)

# # Sheet: DATA INPUT
# worksheet = workbook["DATA INPUT"]

# # Fill the cells, columns 2 - 12
# for i in range(2,12):
#     worksheet.cell(5,i).value = parameters_scrape['Revenue'][i-2]
#     worksheet.cell(6,i).value = parameters_scrape['EPS (Diluted)'][i-2]
#     worksheet.cell(7,i).value = parameters_scrape["Shareholders' Equity"][i-2]
#     worksheet.cell(8,i).value = parameters_scrape["Cash From Operations"][i-2]

# # Fill the cells (11,11), (12,11), (13,11)
# worksheet.cell(11,11).value = parameters_scrape['Total Current Assets'][-1]
# worksheet.cell(12,11).value = parameters_scrape['Total Current Liabilities'][-1]
# worksheet.cell(13,11).value = parameters_scrape['Total Liabilities'][-1]

# # Sheet: ROIC
# worksheet = workbook["ROIC"]

# # Fill the cells, columns 2 - 12
# for i in range(2,12):
#     worksheet.cell(3,i).value = parameters_scrape['Long-Term Debt'][i-2]
    
# # Save the Excel file
# workbook.save("/home/willy/Willy taff/Investing/" + file_name)